In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 30.9 MB/s 
     |████████████████████████████████| 7.6 MB 64.7 MB/s 
     |████████████████████████████████| 182 kB 64.2 MB/s 


# Select data to label

In [1]:
import numpy as np
import pandas as pd
from numpy.random import default_rng

In [2]:
df = pd.read_csv('cybersecurity_incidents_descriptions_v2.csv', encoding="ISO-8859-1")
df.head()

,Unnamed: 0,Description
0,0,September 2022. Iranian hackers targeted Alban...
1,1,September 2022. Hackers targeted Montenegroâ...
2,2,September 2022. Hackers targeted the state -le...
3,3,September 2022. China accused the U.S. Nationa...
4,4,September 2022. The group Anonymous took respo...


In [3]:
random_seed = 2022
num_label = 150

rng = default_rng(random_seed)
idx = rng.choice(len(df), size=num_label, replace=False)
idx

array([765,  86,  46, 726, 898, 671,  14, 858, 525, 843, 441, 573, 694,
        71, 442, 150, 571, 426, 242,  50, 761, 744, 241, 357, 859, 599,
       114, 775, 212, 164, 716, 623,  74, 202, 578,  64, 528, 169, 586,
       507, 757, 682, 511,  42, 769, 912,  92, 370, 883,  77, 312,  17,
        95, 510,   3,  21, 900, 238, 502, 673, 299, 292, 243, 811, 267,
       719, 568, 800, 574, 275,  75,  39, 754, 213, 119, 722, 552, 324,
       518,   2, 261, 734, 144, 394, 407,   0, 838, 555, 246, 487, 894,
       505, 433, 835, 179, 821, 128, 479, 489, 488, 888, 866, 829, 189,
       807, 885, 791, 255, 886, 540, 465, 201, 123, 553, 630, 559, 762,
       808, 637, 284,  90, 424, 395, 435, 893,  68, 913,  97, 354, 834,
       707,  67, 464, 420,  33, 191, 455, 368, 597,  69,  99, 165, 471,
       896, 159, 454, 105, 595, 709, 572])

In [4]:
# unseen data
random_seed = 2022
num_label = 50

rem_idx = [i for i in range(len(df)) if i not in idx]

rng = default_rng(random_seed)
idx_test = rng.choice(rem_idx, size=num_label, replace=False)
idx_test

array([830, 220, 106,  47, 282, 758, 654, 585,  38,  94, 331, 849,  54,
       362, 493, 337, 222, 564,  82, 576, 789, 232,   5, 547, 641, 802,
       527, 801, 231, 905, 842, 293, 647, 608, 897, 863, 706, 313, 411,
       910,  58, 854, 596, 878,  85,  26, 748, 643, 300, 171])

In [5]:
random_seed = 2022
num_label = 50

extra_idx = [i for i in range(len(df)) if i not in idx and i not in idx_test]

rng = default_rng(random_seed)
idx_extra = rng.choice(extra_idx, size=num_label, replace=False)
idx_extra

array([823, 218, 111,  48, 283, 750, 650, 581,  40, 101, 332, 841,  56,
       363, 485, 339, 223, 558,  84, 565, 783, 234,   6, 541, 636, 793,
       519, 794, 901, 903, 836, 294, 644, 603, 891, 857, 700, 315, 412,
       638,  61, 850, 592, 874,  91,  25, 739, 635, 302, 173])

In [ ]:
data_to_label = df.iloc[idx]
data_to_label.to_csv('cyber_incidents_w_labels.csv')

In [ ]:
data_to_label = df.iloc[idx_test]
data_to_label.to_csv('cyber_incidents_w_labels_unseen.csv')

In [ ]:
data_to_label = df.iloc[idx_extra]
data_to_label.to_csv('cyber_incidents_w_labels_extra_train.csv')

# Extractive QnA

In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer

In [ ]:
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

Downloading:   0%|          | 0.00/443 [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-large-uncased-whole-word-masking-finetuned-squad/snapshots/242d9dbb66bb5033025196d5678907307f8fb098/config.json
Model config BertConfig {
  "architectures": [
    "BertForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



Downloading:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--bert-large-uncased-whole-word-masking-finetuned-squad/snapshots/242d9dbb66bb5033025196d5678907307f8fb098/pytorch_model.bin
All model checkpoint weights were used when initializing BertForQuestionAnswering.

All the weights of BertForQuestionAnswering were initialized from the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertForQuestionAnswering for predictions without further training.


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--bert-large-uncased-whole-word-masking-finetuned-squad/snapshots/242d9dbb66bb5033025196d5678907307f8fb098/vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--bert-large-uncased-whole-word-masking-finetuned-squad/snapshots/242d9dbb66bb5033025196d5678907307f8fb098/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-large-uncased-whole-word-masking-finetuned-squad/snapshots/242d9dbb66bb5033025196d5678907307f8fb098/config.json
Model config BertConfig {
  "_name_or_path": "bert-large-uncased-whole-word-masking-finetuned-squad",
  "architectures": [
    "BertForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
model.to(device)

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-12,

In [ ]:
data = pd.read_csv('cybersecurity_incidents_descriptions_v2.csv', encoding = "ISO-8859-1")
data.head()

,Unnamed: 0,Description
0,0,September 2022. Iranian hackers targeted Alban...
1,1,September 2022. Hackers targeted Montenegroâ...
2,2,September 2022. Hackers targeted the state -le...
3,3,September 2022. China accused the U.S. Nationa...
4,4,September 2022. The group Anonymous took respo...


## Extract Features from Descriptions

In [ ]:
# initial attempt
questions = [
    'When did the attack occur?',
    'Which country was the attacker from?',
    'Which country was the victim from?',
    'What type of cyber attack occurred?',
    'Which industry was the victim from?'
]

headers = [
    'time',
    'attacker_origin',
    'victim_origin',
    'type_of_attack',
    'industry'    
]

In [ ]:
questions = [
    'When did the hack occur?',
]

headers = [
    'time',
]

In [ ]:
# get answers from input text
for i, q in enumerate(questions):
  print(i, q)
  a = []

  for text in data.Description.values:
    input_ids = tokenizer.encode(q, text)
    tokens = tokenizer.convert_ids_to_tokens(input_ids)

    #first occurence of [SEP] token
    sep_idx = input_ids.index(tokenizer.sep_token_id)

    #number of tokens in segment A (question) - this will be one more than the sep_idx as the index in Python starts from 0
    num_seg_a = sep_idx+1

    #number of tokens in segment B (text)
    num_seg_b = len(input_ids) - num_seg_a

    #creating the segment ids
    segment_ids = [0]*num_seg_a + [1]*num_seg_b
    #making sure that every input token has a segment id
    assert len(segment_ids) == len(input_ids)

    #token input_ids to represent the input and token segment_ids to differentiate our segments - question and text
    output = model(torch.tensor([input_ids]).to(device),  token_type_ids=torch.tensor([segment_ids]).to(device))

    #tokens with highest start and end scores
    answer_start = torch.argmax(output.start_logits)
    answer_end = torch.argmax(output.end_logits)

    # remove wordpiece tokenization
    answer = tokens[answer_start]
    for j in range(answer_start+1, answer_end+1):
        if tokens[j][0:2] == "##":
            answer += tokens[j][2:]
        else:
            answer += " " + tokens[j]
    a.append(answer)

  data[headers[i]] = a

0 When did the hack occur?


In [ ]:
data.head()

,Unnamed: 0,Description,time
0,0,September 2022. Iranian hackers targeted Alban...,september 2022
1,1,September 2022. Hackers targeted Montenegroâ...,september 2022
2,2,September 2022. Hackers targeted the state -le...,september 2022
3,3,September 2022. China accused the U.S. Nationa...,september 2022
4,4,September 2022. The group Anonymous took respo...,september 2022


In [ ]:
data.to_csv('cybersecurity_incidents_with_extras.csv')

In [ ]:
data.to_csv('cybersecurity_incidents_with_dates.csv')

# Multiple Choice NLP - Hacker Origin

In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 28.5 MB/s 
     |████████████████████████████████| 212 kB 69.4 MB/s 
     |████████████████████████████████| 115 kB 64.4 MB/s 
     |████████████████████████████████| 127 kB 61.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
countries_list = np.load('countries_list.npy')
countries = {}
int_to_countries = {}

for (i, country) in enumerate(countries_list):
  countries[f'country{i}'] = country
  int_to_countries[i] = country

In [ ]:
data_w_labels = pd.read_csv('cyber_incidents_w_labels_v3.csv', encoding="ISO-8859-1", index_col=0)
data_w_labels.head()

,Unnamed: 0.1,Description,Attacker,Victim
765,765,"March 2013. The Syrian Electronic Army, a pro ...",172,0
86,86,January 2022. A Chinese hacking group breached...,36,66
46,46,April 2022. Iranian state tele vision claimed ...,0,81
726,726,September 2014. A false Occupy Central smartph...,36,76
898,898,"September 2007. Francis D elon, Secretary -Gen...",36,62


In [ ]:
data_hacker = data_w_labels[['Description', 'Attacker']]
data_hacker['question'] = 'The hacker was from'
data_hacker.rename(columns={'Attacker': 'label'}, inplace=True)

for k, v in countries.items():
  data_hacker[k] = v

data_hacker.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = fra

,Description,label,question,country0,country1,country2,country3,country4,country5,country6,...,country188,country189,country190,country191,country192,country193,country194,country195,country196,country197
765,"March 2013. The Syrian Electronic Army, a pro ...",172,The hacker was from,Unknown,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,...,United States,Uruguay,Uzbekistan,Vanuatu,Vatican City,Venezuela,Vietnam,Yemen,Zambia,Zimbabwe
86,January 2022. A Chinese hacking group breached...,36,The hacker was from,Unknown,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,...,United States,Uruguay,Uzbekistan,Vanuatu,Vatican City,Venezuela,Vietnam,Yemen,Zambia,Zimbabwe
46,April 2022. Iranian state tele vision claimed ...,0,The hacker was from,Unknown,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,...,United States,Uruguay,Uzbekistan,Vanuatu,Vatican City,Venezuela,Vietnam,Yemen,Zambia,Zimbabwe
726,September 2014. A false Occupy Central smartph...,36,The hacker was from,Unknown,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,...,United States,Uruguay,Uzbekistan,Vanuatu,Vatican City,Venezuela,Vietnam,Yemen,Zambia,Zimbabwe
898,"September 2007. Francis D elon, Secretary -Gen...",36,The hacker was from,Unknown,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,...,United States,Uruguay,Uzbekistan,Vanuatu,Vatican City,Venezuela,Vietnam,Yemen,Zambia,Zimbabwe


In [ ]:
from datasets import Dataset, DatasetDict

In [ ]:
hacker_dataset = Dataset.from_pandas(data_hacker)
#hacker_dataset

In [ ]:
test_split = 0.25

train_dataset, test_dataset = hacker_dataset.train_test_split(test_size=test_split).values()

In [ ]:
h_dataset = DatasetDict({'train': train_dataset,
                         'test': test_dataset})
#h_dataset

### Create DataCollator


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
num_countries = len(countries)
country_keys = countries.keys()

def preprocess_function(examples):
    first_sentences = [[context] * num_countries for context in examples["Description"]]
    #print(first_sentences)
    question_headers = examples["question"]
    second_sentences = [
        [f"{header} {examples[country][i]}" for country in country_keys] for i, header in enumerate(question_headers)
    ]

    first_sentences = sum(first_sentences, [])
    second_sentences = sum(second_sentences, [])

    # print(first_sentences)
    # print(second_sentences)

    tokenized_examples = tokenizer(first_sentences, second_sentences, truncation=True)
    return {k: [v[i : i + num_countries] for i in range(0, len(v), num_countries)] for k, v in tokenized_examples.items()}

In [ ]:
tokenized_h_data = h_dataset.map(preprocess_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import torch

In [ ]:
@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])

        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        return batch

### Train Model - Hacker Origin

In [ ]:
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer

model = AutoModelForMultipleChoice.from_pretrained("bert-base-uncased")

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMultipleChoice: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
model.to(device)

BertForMultipleChoice(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_aff

In [ ]:
# freeze model weights except classification layer
for param in model.parameters():
  param.requires_grad = False

for param in model.classifier.parameters():
  param.requires_grad = True

for param in model.bert.encoder.layer[11].parameters():
  param.requires_grad = True

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=2,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_h_data["train"],
    eval_dataset=tokenized_h_data["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer=tokenizer),
)

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: country75, country29, country107, country89, country92, question, country183, country53, country95, country27, country137, country42, country99, country88, country93, country180, country70, country97, country122, country135, country177, country163, country126, country110, country96, country133, country50, country74, country49, country59, country175, country51, country63, country22, country119, country150, country162, country4, country165, country140, country146, country139, Description, country124, country101, country94, country35, country21, country116, country

Epoch,Training Loss,Validation Loss
1,No log,1.293494
2,No log,1.253681


The following columns in the evaluation set don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: country75, country29, country107, country89, country92, question, country183, country53, country95, country27, country137, country42, country99, country88, country93, country180, country70, country97, country122, country135, country177, country163, country126, country110, country96, country133, country50, country74, country49, country59, country175, country51, country63, country22, country119, country150, country162, country4, country165, country140, country146, country139, Description, country124, country101, country94, country35, country21, country116, country132, country68, country164, country190, country16, country13, country28, country64, country129, country158, country170, country188, country10, country145, country61, country159, country189, country176, country193, country24, country114, country108, country40, country26, country14, country128, 

TrainOutput(global_step=112, training_loss=0.09074389083044869, metrics={'train_runtime': 260.521, 'train_samples_per_second': 0.86, 'train_steps_per_second': 0.43, 'total_flos': 1591355374535520.0, 'train_loss': 0.09074389083044869, 'epoch': 2.0})

In [ ]:
# save model weights
torch.save(model.state_dict(), 'Bert_MC_hacker_0.92.pt')

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp Bert_MC_hacker_0.92.pt /content/drive/MyDrive/EDAV\ Final\ Project

### Validate Accuracy on validation set

In [ ]:
output = trainer.predict(tokenized_h_data['test'])

NameError: ignored

In [ ]:
logits = torch.tensor(output.predictions)
y_preds = torch.nn.Softmax(dim=1)(logits)
y_preds = torch.argmax(y_preds, dim=1)

In [ ]:
y_true = tokenized_h_data['test']['label']

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(y_true, y_preds)

0.9210526315789473

### Label Entire Dataset

In [ ]:
saved_model_file = '/content/drive/MyDrive/EDAV Final Project/Bert_MC_hacker_0.92.pt'

In [ ]:
model.load_state_dict(torch.load(saved_model_file))

<All keys matched successfully>

In [ ]:
raw_data = pd.read_csv('cybersecurity_incidents_descriptions_v2.csv', encoding = "ISO-8859-1")
raw_data['question'] = 'The hacker was from'
for k, v in countries.items():
  raw_data[k] = v

raw_data.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  after removing the cwd from sys.path.


,Unnamed: 0,Description,question,country0,country1,country2,country3,country4,country5,country6,...,country188,country189,country190,country191,country192,country193,country194,country195,country196,country197
0,0,September 2022. Iranian hackers targeted Alban...,The hacker was from,Unknown,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,...,United States,Uruguay,Uzbekistan,Vanuatu,Vatican City,Venezuela,Vietnam,Yemen,Zambia,Zimbabwe
1,1,September 2022. Hackers targeted Montenegroâ...,The hacker was from,Unknown,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,...,United States,Uruguay,Uzbekistan,Vanuatu,Vatican City,Venezuela,Vietnam,Yemen,Zambia,Zimbabwe
2,2,September 2022. Hackers targeted the state -le...,The hacker was from,Unknown,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,...,United States,Uruguay,Uzbekistan,Vanuatu,Vatican City,Venezuela,Vietnam,Yemen,Zambia,Zimbabwe
3,3,September 2022. China accused the U.S. Nationa...,The hacker was from,Unknown,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,...,United States,Uruguay,Uzbekistan,Vanuatu,Vatican City,Venezuela,Vietnam,Yemen,Zambia,Zimbabwe
4,4,September 2022. The group Anonymous took respo...,The hacker was from,Unknown,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,...,United States,Uruguay,Uzbekistan,Vanuatu,Vatican City,Venezuela,Vietnam,Yemen,Zambia,Zimbabwe


In [ ]:
r_dataset = Dataset.from_pandas(raw_data)
tokenized_r_data = r_dataset.map(preprocess_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
@dataclass
class DataCollatorForMCPrediction:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        #label_name = "label" if "label" in features[0].keys() else "labels"
        #labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])

        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        #batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        return batch

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=30,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_r_data,
    eval_dataset=tokenized_r_data,
    tokenizer=tokenizer,
    data_collator=DataCollatorForMCPrediction(tokenizer=tokenizer),
)

In [ ]:
with torch.no_grad():
  model.eval()
  all_labels = trainer.predict(tokenized_r_data)

The following columns in the test set don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: country26, country159, country189, country89, country144, country121, country176, country192, country156, country196, country100, country135, country92, country137, country145, country50, country109, country164, country148, country78, country128, country143, country163, country16, country150, country86, country126, country24, country118, country83, country183, country1, country166, country12, country85, country60, country57, country157, country37, country172, country11, country13, country138, country54, country114, country2, country122, country44, country66, country113, country149, country64, country30, country23, country136, country169, country110, country153, country97, country127, country3, country18, country139, country20, country123, country154, country160, country98, country195, country36, country43, country22, country111, country105, country5, count

In [ ]:
all_labels

PredictionOutput(predictions=array([[ -6.0614295, -13.067052 ,  -9.710693 , ..., -13.576829 ,
        -14.927971 , -14.973624 ],
       [ -2.235025 , -14.497287 , -13.79994  , ..., -14.985605 ,
        -15.244015 , -15.247271 ],
       [  9.583682 , -14.687954 , -14.137086 , ..., -14.69092  ,
        -14.901741 , -14.765105 ],
       ...,
       [ -3.707081 , -13.832757 , -14.732    , ..., -14.086665 ,
        -14.518508 , -14.595004 ],
       [ -3.034494 , -12.86735  , -14.191491 , ..., -13.237676 ,
        -14.027038 , -13.927505 ],
       [  3.4076777, -11.758878 , -14.353501 , ..., -13.452581 ,
        -14.166995 , -14.285461 ]], dtype=float32), label_ids=None, metrics={'test_runtime': 616.6546, 'test_samples_per_second': 1.482, 'test_steps_per_second': 0.741})

In [ ]:
logits = torch.tensor(all_labels.predictions)
all_preds = torch.nn.Softmax(dim=1)(logits)
all_preds = torch.argmax(all_preds, dim=1)
all_preds

tensor([ 81, 146,   0,  36,   0,   0, 146,   0,   0, 146, 146, 146,   0, 146,
        146,   0,  36,   0,   0,  36,   0, 146,   0, 146,  36, 146,  36,  81,
          0,  36,  36,  81,  36,  92,   0,   0, 146,  81,  81,   0,   0,  36,
          0, 146, 146, 146,   0, 146,   0,  92,   0,  92,   0,   0,   0,   0,
         81, 146, 146, 146,   0,   0, 135, 146,   0, 146, 188,  36, 146,   0,
          0,  36,   0,  92, 146,  36, 146,  36, 135,  81, 146,   0,   0,  92,
         36,   0,  36,  92, 146,   0,   0,  36,  81,   0,   0, 146,   0,  36,
          0,  36,  36,  36, 146, 146,   0,   0,  36,   0,   0,   0,  36,   0,
         81,  80,   0, 146,  36, 146, 146,  81,  36,   0,  92,   0,  36,   0,
         36,   0,   0, 146, 146,  36,   0,  36,   0,   0,  36,  36,   0,   0,
          0,  36,  81,   0,  36,   0, 146, 146, 146,  36,  81,  36, 146, 146,
        146, 146, 146,   0,   0, 146,   0,  92, 146,   0,  92, 146,   0, 146,
          0,   0,  36,   0,  36,   0, 146,   0,  36,   0, 146,  

In [ ]:
labeled_data = pd.DataFrame(raw_data['Description'])
labeled_data['hacker_origin_class'] = all_preds
labeled_data['hacker_origin'] = labeled_data['hacker_origin_class'].apply(lambda x: int_to_countries[x])

In [ ]:
labeled_data.head()

,Description,hacker_origin_class,hacker_origin
0,September 2022. Iranian hackers targeted Alban...,81,Iran
1,September 2022. Hackers targeted Montenegroâ...,146,Russia
2,September 2022. Hackers targeted the state -le...,0,Unknown
3,September 2022. China accused the U.S. Nationa...,36,China
4,September 2022. The group Anonymous took respo...,0,Unknown


In [ ]:
labeled_data.to_csv('cyber_incidents_w_hack_origin.csv')

### Compare to previously labeled train and test data

In [ ]:
data_hacker = pd.read_csv('cyber_incidents_w_labels_v3.csv', index_col=0)
data_hacker

,Unnamed: 0.1,Description,Attacker,Victim
765,765,"March 2013. The Syrian Electronic Army, a pro ...",172,0
86,86,January 2022. A Chinese hacking group breached...,36,66
46,46,April 2022. Iranian state tele vision claimed ...,0,81
726,726,September 2014. A false Occupy Central smartph...,36,76
898,898,"September 2007. Francis D elon, Secretary -Gen...",36,62
...,...,...,...,...
454,454,January 2019. France attributed a cyberattack ...,146,62
105,105,November 2021. Chinese officials claim a forei...,0,36
595,595,August 2017 . Researchers inform the Estonian ...,0,57
709,709,March 2015. An Iranian hacking group conducted...,81,0


In [ ]:
results = pd.concat([labeled_data, data_hacker], axis=1)[['Description', 'hacker_origin_class', 'hacker_origin', 'Attacker']]

In [ ]:
results.to_csv('cyber_incidents_w_hack_origin.csv')

In [ ]:
# mislabeled data
results[results['hacker_origin_class'] != results['Attacker']].dropna()

Description            8
Description            8
hacker_origin_class    8
hacker_origin          8
Attacker               8
dtype: int64

### Compare Unseen Data

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
labeled_hacker = pd.read_csv('cyber_incidents_w_hack_origin_v2.csv', index_col=0)
labeled_hacker.head()

,Description,Description.1,hacker_origin_class,hacker_origin,Attacker
0,September 2022. Iranian hackers targeted Alban...,September 2022. Iranian hackers targeted Alban...,81,Iran,81.0
1,September 2022. Hackers targeted Montenegroâ...,NaN,146,Russia,NaN
2,September 2022. Hackers targeted the state -le...,September 2022. Hackers targeted the state -le...,0,Unknown,0.0
3,September 2022. China accused the U.S. Nationa...,September 2022. China accused the U.S. Nationa...,36,China,188.0
4,September 2022. The group Anonymous took respo...,NaN,0,Unknown,NaN


In [ ]:
unseen_data = pd.read_csv('cyber_incidents_w_labels_unseen.csv', index_col=0)
unseen_data.head()

,Unnamed: 0.1,Description,hacker_label,victim_label
830,830,December 2010. British Foreign Minister Willia...,0,187
220,220,"December 2020. On Christmas Eve, hackers hit t...",0,187
106,106,November 2021 . After CISA publicly shared det...,36,0
47,47,April 2022 . Russian hackers targeted the Cost...,146,41
282,282,September 2020. Norway announced it had defend...,146,133


In [ ]:
hacker_data = pd.concat([labeled_hacker, unseen_data], axis=1)
hacker_data.head()

,Description,Description.1,hacker_origin_class,hacker_origin,Attacker,Unnamed: 0.1,Description,hacker_label,victim_label
0,September 2022. Iranian hackers targeted Alban...,September 2022. Iranian hackers targeted Alban...,81,Iran,81.0,NaN,NaN,NaN,NaN
1,September 2022. Hackers targeted Montenegroâ...,NaN,146,Russia,NaN,NaN,NaN,NaN,NaN
2,September 2022. Hackers targeted the state -le...,September 2022. Hackers targeted the state -le...,0,Unknown,0.0,NaN,NaN,NaN,NaN
3,September 2022. China accused the U.S. Nationa...,September 2022. China accused the U.S. Nationa...,36,China,188.0,NaN,NaN,NaN,NaN
4,September 2022. The group Anonymous took respo...,NaN,0,Unknown,NaN,NaN,NaN,NaN,NaN


In [ ]:
# check accuracy
hacker_data[hacker_data['hacker_origin_class'] != hacker_data['hacker_label']][['Description', 'hacker_origin_class', 'hacker_label']].dropna().count()

Description            9
Description            9
hacker_origin_class    9
hacker_label           9
dtype: int64

In [ ]:
hacker_data.to_csv('hacker_origin_w_unseen_labels.csv')

# Multiple Choice NLP - Victim Origin

In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 24.2 MB/s 
     |████████████████████████████████| 212 kB 63.4 MB/s 
     |████████████████████████████████| 115 kB 37.8 MB/s 
     |████████████████████████████████| 127 kB 55.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
countries_list = np.load('countries_list.npy')
countries = {}
int_to_countries = {}

for (i, country) in enumerate(countries_list):
  countries[f'country{i}'] = country
  int_to_countries[i] = country

In [ ]:
data_w_labels = pd.read_csv('cyber_incidents_w_labels_v3.csv', encoding="ISO-8859-1", index_col=0)
data_w_labels.head()

,Unnamed: 0.1,Description,Attacker,Victim
765,765,"March 2013. The Syrian Electronic Army, a pro ...",172,0
86,86,January 2022. A Chinese hacking group breached...,36,66
46,46,April 2022. Iranian state tele vision claimed ...,0,81
726,726,September 2014. A false Occupy Central smartph...,36,76
898,898,"September 2007. Francis D elon, Secretary -Gen...",36,62


In [ ]:
current_question = 'The victim of the hack was from'

In [ ]:
data_victim = data_w_labels[['Description', 'Victim']]
data_victim['question'] = current_question
data_victim.rename(columns={'Victim': 'label'}, inplace=True)

for k, v in countries.items():
  data_victim[k] = v

data_victim.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = fra

,Description,label,question,country0,country1,country2,country3,country4,country5,country6,...,country188,country189,country190,country191,country192,country193,country194,country195,country196,country197
765,"March 2013. The Syrian Electronic Army, a pro ...",0,The victim of the hack was from,Unknown,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,...,United States,Uruguay,Uzbekistan,Vanuatu,Vatican City,Venezuela,Vietnam,Yemen,Zambia,Zimbabwe
86,January 2022. A Chinese hacking group breached...,66,The victim of the hack was from,Unknown,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,...,United States,Uruguay,Uzbekistan,Vanuatu,Vatican City,Venezuela,Vietnam,Yemen,Zambia,Zimbabwe
46,April 2022. Iranian state tele vision claimed ...,81,The victim of the hack was from,Unknown,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,...,United States,Uruguay,Uzbekistan,Vanuatu,Vatican City,Venezuela,Vietnam,Yemen,Zambia,Zimbabwe
726,September 2014. A false Occupy Central smartph...,76,The victim of the hack was from,Unknown,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,...,United States,Uruguay,Uzbekistan,Vanuatu,Vatican City,Venezuela,Vietnam,Yemen,Zambia,Zimbabwe
898,"September 2007. Francis D elon, Secretary -Gen...",62,The victim of the hack was from,Unknown,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,...,United States,Uruguay,Uzbekistan,Vanuatu,Vatican City,Venezuela,Vietnam,Yemen,Zambia,Zimbabwe


In [ ]:
from datasets import Dataset, DatasetDict

In [ ]:
victim_dataset = Dataset.from_pandas(data_victim)
#victim_dataset

In [ ]:
test_split = 0.25

train_dataset, test_dataset = victim_dataset.train_test_split(test_size=test_split).values()

In [ ]:
v_dataset = DatasetDict({'train': train_dataset,
                         'test': test_dataset})
#v_dataset

### Create DataCollator


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
num_countries = len(countries)
country_keys = countries.keys()

def preprocess_function(examples):
    first_sentences = [[context] * num_countries for context in examples["Description"]]
    #print(first_sentences)
    question_headers = examples["question"]
    second_sentences = [
        [f"{header} {examples[country][i]}" for country in country_keys] for i, header in enumerate(question_headers)
    ]

    first_sentences = sum(first_sentences, [])
    second_sentences = sum(second_sentences, [])

    # print(first_sentences)
    # print(second_sentences)

    tokenized_examples = tokenizer(first_sentences, second_sentences, truncation=True)
    return {k: [v[i : i + num_countries] for i in range(0, len(v), num_countries)] for k, v in tokenized_examples.items()}

In [ ]:
tokenized_v_data = v_dataset.map(preprocess_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import torch

In [ ]:
@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])

        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        return batch

### Train Model - Victim Origin

In [ ]:
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer

model = AutoModelForMultipleChoice.from_pretrained("bert-base-uncased")

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMultipleChoice: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly

Load weights (if retraining)

In [ ]:
saved_model_file = '/content/drive/MyDrive/EDAV Final Project/Bert_MC_victim_0.76.pt'

In [ ]:
model.load_state_dict(torch.load(saved_model_file))

<All keys matched successfully>

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
model.to(device)

BertForMultipleChoice(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_aff

In [ ]:
# freeze model weights except classification layer and last BERT layer
for param in model.parameters():
  param.requires_grad = False

for param in model.classifier.parameters():
  param.requires_grad = True

for param in model.bert.encoder.layer[11].parameters():
  param.requires_grad = True

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=10,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_v_data["train"],
    eval_dataset=tokenized_v_data["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer=tokenizer),
)

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: country62, country99, country64, country28, country196, country34, country0, country14, country76, country124, country48, country106, country6, country51, country144, country87, country68, country38, country102, country122, country112, country195, country189, country74, country107, country45, country111, country129, country83, country181, country130, country162, country142, country73, country125, country169, country191, country187, country153, country117, country141, country71, country63, country65, country140, country67, country121, country7, country90, country

Epoch,Training Loss,Validation Loss
1,No log,0.960495
2,No log,1.120193
3,No log,0.899321
4,No log,0.942555
5,No log,0.940939
6,No log,0.935303
7,No log,1.003453
8,No log,1.055461
9,0.245400,1.115129
10,0.245400,1.277554


The following columns in the evaluation set don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: country62, country99, country64, country28, country196, country34, country0, country14, country76, country124, country48, country106, country6, country51, country144, country87, country68, country38, country102, country122, country112, country195, country189, country74, country107, country45, country111, country129, country83, country181, country130, country162, country142, country73, country125, country169, country191, country187, country153, country117, country141, country71, country63, country65, country140, country67, country121, country7, country90, country143, country19, country75, country114, country35, country17, country44, country80, country70, country156, country61, country131, country172, country159, country30, __index_level_0__, country123, country81, country108, country118, country133, country101, country56, country157, country41, countr

TrainOutput(global_step=560, training_loss=0.2386921967778887, metrics={'train_runtime': 1290.7762, 'train_samples_per_second': 0.868, 'train_steps_per_second': 0.434, 'total_flos': 8304758674526160.0, 'train_loss': 0.2386921967778887, 'epoch': 10.0})

In [ ]:
# save model weights
torch.save(model.state_dict(), 'Bert_MC_victim_0.82.pt')

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp Bert_MC_victim_0.82.pt /content/drive/MyDrive/EDAV\ Final\ Project

### Validate Accuracy on validation set

In [ ]:
output = trainer.predict(tokenized_v_data['test'])

The following columns in the test set don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: country62, country99, country64, country28, country196, country34, country0, country14, country76, country124, country48, country106, country6, country51, country144, country87, country68, country38, country102, country122, country112, country195, country189, country74, country107, country45, country111, country129, country83, country181, country130, country162, country142, country73, country125, country169, country191, country187, country153, country117, country141, country71, country63, country65, country140, country67, country121, country7, country90, country143, country19, country75, country114, country35, country17, country44, country80, country70, country156, country61, country131, country172, country159, country30, __index_level_0__, country123, country81, country108, country118, country133, country101, country56, country157, country41, country126, 

In [ ]:
logits = torch.tensor(output.predictions)
y_preds = torch.nn.Softmax(dim=1)(logits)
y_preds = torch.argmax(y_preds, dim=1)

In [ ]:
y_true = tokenized_v_data['test']['label']

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(y_true, y_preds)

0.8157894736842105

### Label Entire Dataset

In [ ]:
saved_model_file = '/content/drive/MyDrive/EDAV Final Project/Bert_MC_victim_.pt'

In [ ]:
model.load_state_dict(torch.load(saved_model_file))

<All keys matched successfully>

In [ ]:
raw_data = pd.read_csv('cybersecurity_incidents_descriptions_v2.csv', encoding = "ISO-8859-1")
raw_data['question'] = current_question
for k, v in countries.items():
  raw_data[k] = v

raw_data.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  after removing the cwd from sys.path.


,Unnamed: 0,Description,question,country0,country1,country2,country3,country4,country5,country6,...,country188,country189,country190,country191,country192,country193,country194,country195,country196,country197
0,0,September 2022. Iranian hackers targeted Alban...,The victim of the hack was from,Unknown,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,...,United States,Uruguay,Uzbekistan,Vanuatu,Vatican City,Venezuela,Vietnam,Yemen,Zambia,Zimbabwe
1,1,September 2022. Hackers targeted Montenegroâ...,The victim of the hack was from,Unknown,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,...,United States,Uruguay,Uzbekistan,Vanuatu,Vatican City,Venezuela,Vietnam,Yemen,Zambia,Zimbabwe
2,2,September 2022. Hackers targeted the state -le...,The victim of the hack was from,Unknown,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,...,United States,Uruguay,Uzbekistan,Vanuatu,Vatican City,Venezuela,Vietnam,Yemen,Zambia,Zimbabwe
3,3,September 2022. China accused the U.S. Nationa...,The victim of the hack was from,Unknown,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,...,United States,Uruguay,Uzbekistan,Vanuatu,Vatican City,Venezuela,Vietnam,Yemen,Zambia,Zimbabwe
4,4,September 2022. The group Anonymous took respo...,The victim of the hack was from,Unknown,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,...,United States,Uruguay,Uzbekistan,Vanuatu,Vatican City,Venezuela,Vietnam,Yemen,Zambia,Zimbabwe


In [ ]:
r_dataset = Dataset.from_pandas(raw_data)
tokenized_r_data = r_dataset.map(preprocess_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
@dataclass
class DataCollatorForMCPrediction:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        #label_name = "label" if "label" in features[0].keys() else "labels"
        #labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])

        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        #batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        return batch

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=30,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_r_data,
    eval_dataset=tokenized_r_data,
    tokenizer=tokenizer,
    data_collator=DataCollatorForMCPrediction(tokenizer=tokenizer),
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
with torch.no_grad():
  model.eval()
  all_labels = trainer.predict(tokenized_r_data)

The following columns in the test set don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: country62, country99, country64, country28, country196, country34, country0, country14, country76, country124, country48, country106, country6, country51, country144, country87, country68, country38, country102, country122, country112, country195, country189, country74, country107, country45, country111, country129, country83, country181, country130, country162, country142, country73, country125, country169, country191, country187, country153, country117, country141, country71, country63, country65, country140, country67, country121, country7, country90, country143, country19, country75, country114, country35, country17, country44, country80, country70, country156, country61, country131, country172, country159, country30, country123, country81, country108, country118, country133, country101, country56, country157, country41, country126, country31, country1

In [ ]:
all_labels

PredictionOutput(predictions=array([[ -9.697091 , -10.932494 ,  10.133365 , ..., -11.082958 ,
        -11.113456 , -11.098981 ],
       [ -6.065379 , -11.091463 ,  -9.598735 , ..., -11.135471 ,
        -11.120323 , -11.12735  ],
       [ -5.0877023, -11.060246 , -10.933496 , ..., -11.108023 ,
        -11.1032   , -11.111873 ],
       ...,
       [ -5.7409534, -10.968149 , -11.080103 , ..., -11.000016 ,
        -11.026137 , -11.038159 ],
       [ -7.2895503,  -9.099486 , -10.924352 , ..., -10.988972 ,
        -10.983445 , -11.021151 ],
       [ -7.7096586, -10.892918 , -11.111534 , ..., -11.091549 ,
        -11.067061 , -11.053409 ]], dtype=float32), label_ids=None, metrics={'test_runtime': 650.1999, 'test_samples_per_second': 1.406, 'test_steps_per_second': 0.703})

In [ ]:
logits = torch.tensor(all_labels.predictions)
all_preds = torch.nn.Softmax(dim=1)(logits)
all_preds = torch.argmax(all_preds, dim=1)
all_preds

tensor([  2, 120,  22,  36,  81, 115, 187,  85, 173,  61, 185,  98,  68, 187,
        120,  57,   0,  81,  36,  17, 187, 104,   2, 185,  36, 104,   0,  84,
         81,  36,   9,  84,   9,   0, 188,  66, 133, 187,  88,  59,  47, 188,
        146,  85, 145, 142,  81,  41,   0,  93, 165, 188,  61,  84, 185,  79,
        188, 185, 188, 185, 185, 112,  79, 185,  47, 146,  36,  62, 188, 146,
         32, 188,  84,   0, 185,  36, 185,  36,  79,   0, 188,  66,   0,   0,
         36, 187,  66,  93,  32,   4,  68,   9,   0, 185,  84, 146,  17,   0,
         79,   9,   0, 188,   9, 146, 188,  36,   0, 187, 188, 188,   0,  81,
         84,  80,  84, 146,   1, 146, 188,  84, 104,  77, 188,   0, 133,   0,
          9,  16,  81, 160, 146,  84,  85,   0,  57,   0, 141,   0, 164,  79,
          0, 188, 188, 146, 133,  81, 146, 146, 146,   1,  84, 188, 146, 187,
        188, 142, 188, 187, 188, 127, 188,  93, 188,  87,  93,  83,  41, 188,
        133,  17, 146, 142,   0, 188, 185,  84, 188, 187, 170, 1

In [ ]:
labeled_data = pd.DataFrame(raw_data['Description'])
labeled_data['victim_origin_class'] = all_preds
labeled_data['victim_origin'] = labeled_data['victim_origin_class'].apply(lambda x: int_to_countries[x])

In [ ]:
labeled_data.head()

,Description,victim_origin_class,victim_origin
0,September 2022. Iranian hackers targeted Alban...,2,Albania
1,September 2022. Hackers targeted Montenegroâ...,120,Montenegro
2,September 2022. Hackers targeted the state -le...,22,Bosnia and Herzegovina
3,September 2022. China accused the U.S. Nationa...,36,China
4,September 2022. The group Anonymous took respo...,81,Iran


In [ ]:
labeled_data.to_csv('cyber_incidents_w_victim_origin.csv')

### Compare to previously labeled train and test data

In [ ]:
data_victim = pd.read_csv('cyber_incidents_w_labels_v3.csv', index_col=0)
data_victim

,Unnamed: 0.1,Description,Attacker,Victim
765,765,"March 2013. The Syrian Electronic Army, a pro ...",172,0
86,86,January 2022. A Chinese hacking group breached...,36,66
46,46,April 2022. Iranian state tele vision claimed ...,0,81
726,726,September 2014. A false Occupy Central smartph...,36,76
898,898,"September 2007. Francis D elon, Secretary -Gen...",36,62
...,...,...,...,...
454,454,January 2019. France attributed a cyberattack ...,146,62
105,105,November 2021. Chinese officials claim a forei...,0,36
595,595,August 2017 . Researchers inform the Estonian ...,0,57
709,709,March 2015. An Iranian hacking group conducted...,81,0


In [ ]:
results = pd.concat([labeled_data, data_victim], axis=1)[['Description', 'victim_origin_class', 'victim_origin', 'Victim']]

In [ ]:
results.to_csv('cyber_incidents_w_victim_origin.csv')

In [ ]:
# mislabeled data
results[results['victim_origin_class'] != results['Victim']].dropna().count()

Description            8
Description            8
victim_origin_class    8
victim_origin          8
Victim                 8
dtype: int64

### Compare Unseen Data

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
labeled_victim = pd.read_csv('cyber_incidents_w_victim_origin.csv', index_col=0)
labeled_victim.head()

,Description,Description.1,victim_origin_class,victim_origin,Victim
0,September 2022. Iranian hackers targeted Alban...,September 2022. Iranian hackers targeted Alban...,2,Albania,2.0
1,September 2022. Hackers targeted Montenegroâ...,NaN,120,Montenegro,NaN
2,September 2022. Hackers targeted the state -le...,September 2022. Hackers targeted the state -le...,22,Bosnia and Herzegovina,22.0
3,September 2022. China accused the U.S. Nationa...,September 2022. China accused the U.S. Nationa...,36,China,36.0
4,September 2022. The group Anonymous took respo...,NaN,81,Iran,NaN


In [ ]:
unseen_data = pd.read_csv('cyber_incidents_w_labels_unseen.csv', index_col=0)
unseen_data.head()

,Unnamed: 0.1,Description,hacker_label,victim_label
830,830,December 2010. British Foreign Minister Willia...,0,187
220,220,"December 2020. On Christmas Eve, hackers hit t...",0,187
106,106,November 2021 . After CISA publicly shared det...,36,0
47,47,April 2022 . Russian hackers targeted the Cost...,146,41
282,282,September 2020. Norway announced it had defend...,146,133


In [ ]:
victim_data = pd.concat([labeled_victim, unseen_data], axis=1)
victim_data.head()

,Description,Description.1,victim_origin_class,victim_origin,Victim,Unnamed: 0.1,Description,hacker_label,victim_label
0,September 2022. Iranian hackers targeted Alban...,September 2022. Iranian hackers targeted Alban...,2,Albania,2.0,NaN,NaN,NaN,NaN
1,September 2022. Hackers targeted Montenegroâ...,NaN,120,Montenegro,NaN,NaN,NaN,NaN,NaN
2,September 2022. Hackers targeted the state -le...,September 2022. Hackers targeted the state -le...,22,Bosnia and Herzegovina,22.0,NaN,NaN,NaN,NaN
3,September 2022. China accused the U.S. Nationa...,September 2022. China accused the U.S. Nationa...,36,China,36.0,NaN,NaN,NaN,NaN
4,September 2022. The group Anonymous took respo...,NaN,81,Iran,NaN,NaN,NaN,NaN,NaN


In [ ]:
# check accuracy
victim_data[victim_data['victim_origin_class'] != victim_data['victim_label']][['Description', 'victim_origin_class', 'victim_label']].dropna().count()

Description            6
Description            6
victim_origin_class    6
victim_label           6
dtype: int64

In [ ]:
victim_data.to_csv('victim_origin_w_unseen_labels.csv')

# Multiple Choice NLP - Industry

In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 15.0 MB/s 
     |████████████████████████████████| 212 kB 54.1 MB/s 
     |████████████████████████████████| 115 kB 57.2 MB/s 
     |████████████████████████████████| 127 kB 68.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
industries_list = np.load('industries_list.npy')
industries = {}
int_to_industries = {}

for (i, industry) in enumerate(industries_list):
  industries[f'industry{i}'] = industry
  int_to_industries[i] = industry

In [ ]:
data_w_labels = pd.read_csv('cyber_incidents_w_labels_v3.csv', encoding="ISO-8859-1", index_col=0)
data_w_labels.head()

,Unnamed: 0.1,Description,Attacker,Victim,Industry
765,765,"March 2013. The Syrian Electronic Army, a pro ...",172,0,9
86,86,January 2022. A Chinese hacking group breached...,36,66,1
46,46,April 2022. Iranian state tele vision claimed ...,0,81,6
726,726,September 2014. A false Occupy Central smartph...,36,76,0
898,898,"September 2007. Francis D elon, Secretary -Gen...",36,62,2


In [ ]:
current_question = 'The industry targeted was'

In [ ]:
data_ids = data_w_labels[['Description', 'Industry']]
data_ids['question'] = current_question
data_ids.rename(columns={'Industry': 'label'}, inplace=True)

for k, v in industries.items():
  data_ids[k] = v

data_ids.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

,Description,label,question,industry0,industry1,industry2,industry3,industry4,industry5,industry6,industry7,industry8,industry9,industry10,industry11,industry12,industry13
765,"March 2013. The Syrian Electronic Army, a pro ...",9,The industry targeted was,Unknown,Telecommunications,Defense,Education,Energy,Finance,Government,Healthcare,Manufacturing,Media,Non-profit,Technology,Transportation,Utilities
86,January 2022. A Chinese hacking group breached...,1,The industry targeted was,Unknown,Telecommunications,Defense,Education,Energy,Finance,Government,Healthcare,Manufacturing,Media,Non-profit,Technology,Transportation,Utilities
46,April 2022. Iranian state tele vision claimed ...,6,The industry targeted was,Unknown,Telecommunications,Defense,Education,Energy,Finance,Government,Healthcare,Manufacturing,Media,Non-profit,Technology,Transportation,Utilities
726,September 2014. A false Occupy Central smartph...,0,The industry targeted was,Unknown,Telecommunications,Defense,Education,Energy,Finance,Government,Healthcare,Manufacturing,Media,Non-profit,Technology,Transportation,Utilities
898,"September 2007. Francis D elon, Secretary -Gen...",2,The industry targeted was,Unknown,Telecommunications,Defense,Education,Energy,Finance,Government,Healthcare,Manufacturing,Media,Non-profit,Technology,Transportation,Utilities


In [ ]:
from datasets import Dataset, DatasetDict

In [ ]:
ids_dataset = Dataset.from_pandas(data_ids)
#ids_dataset

In [ ]:
test_split = 0.25

train_dataset, test_dataset = ids_dataset.train_test_split(test_size=test_split).values()

In [ ]:
i_dataset = DatasetDict({'train': train_dataset,
                         'test': test_dataset})
#i_dataset

### Create DataCollator


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.24.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/vocab.txt
loading file to

In [ ]:
num_industries = len(industries)
industries_keys = industries.keys()

def preprocess_function(examples):
    first_sentences = [[context] * num_industries for context in examples["Description"]]
    #print(first_sentences)
    question_headers = examples["question"]
    second_sentences = [
        [f"{header} {examples[industry][i]}" for industry in industries_keys] for i, header in enumerate(question_headers)
    ]

    first_sentences = sum(first_sentences, [])
    second_sentences = sum(second_sentences, [])

    # print(first_sentences)
    # print(second_sentences)

    tokenized_examples = tokenizer(first_sentences, second_sentences, truncation=True)
    return {k: [v[i : i + num_industries] for i in range(0, len(v), num_industries)] for k, v in tokenized_examples.items()}

In [ ]:
tokenized_i_data = i_dataset.map(preprocess_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import torch

In [ ]:
@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])

        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        return batch

### Train Model - Industry Hacked


In [ ]:
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer

model = AutoModelForMultipleChoice.from_pretrained("bert-base-uncased")

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.24.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/pytorch_m

Load saved weights (if retraining)

In [ ]:
saved_model_file = '/content/drive/MyDrive/EDAV Final Project/Bert_MC_ids_.pt'

In [ ]:
model.load_state_dict(torch.load(saved_model_file))

<All keys matched successfully>

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
model.to(device)

BertForMultipleChoice(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_aff

In [ ]:
# freeze model weights except classification layer and last BERT layer
for param in model.parameters():
  param.requires_grad = False

for param in model.classifier.parameters():
  param.requires_grad = True

for param in model.bert.encoder.layer[11:].parameters():
  param.requires_grad = True

In [ ]:
for param in model.parameters():
  param.requires_grad = True

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=1.5e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=15,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_i_data["train"],
    eval_dataset=tokenized_i_data["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer=tokenizer),
)

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: industry6, industry1, industry3, industry11, __index_level_0__, industry7, industry9, question, Description, industry8, industry5, industry2, industry4, industry10, industry12, industry0, industry13. If industry6, industry1, industry3, industry11, __index_level_0__, industry7, industry9, question, Description, industry8, industry5, industry2, industry4, industry10, industry12, industry0, industry13 are not expected by `BertForMultipleChoice.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: Fut

Epoch,Training Loss,Validation Loss
1,No log,1.186778
2,No log,1.064656
3,No log,0.960535
4,No log,1.176923
5,No log,0.938655
6,No log,1.020402
7,No log,1.069973
8,No log,0.829362
9,0.001900,0.950274
10,0.001900,1.494746


The following columns in the evaluation set don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: industry6, industry1, industry3, industry11, __index_level_0__, industry7, industry9, question, Description, industry8, industry5, industry2, industry4, industry10, industry12, industry0, industry13. If industry6, industry1, industry3, industry11, __index_level_0__, industry7, industry9, question, Description, industry8, industry5, industry2, industry4, industry10, industry12, industry0, industry13 are not expected by `BertForMultipleChoice.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 38
  Batch size = 2
The following columns in the evaluation set don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: industry6, industry1, industry3, industry11, __index_level_0__, industry7, industry9, question, Description, industry8, industry5, industry2, industry4, industry10,

TrainOutput(global_step=840, training_loss=0.0011692836237627835, metrics={'train_runtime': 324.8815, 'train_samples_per_second': 5.171, 'train_steps_per_second': 2.586, 'total_flos': 777856184390880.0, 'train_loss': 0.0011692836237627835, 'epoch': 15.0})

In [ ]:
# save model weights
torch.save(model.state_dict(), 'Bert_MC_ids_0.81.pt')

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp Bert_MC_ids_0.81.pt /content/drive/MyDrive/EDAV\ Final\ Project

### Validate Accuracy on validation set

In [ ]:
output = trainer.predict(tokenized_i_data['test'])

The following columns in the test set don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: industry6, industry1, industry3, industry11, __index_level_0__, industry7, industry9, question, Description, industry8, industry5, industry2, industry4, industry10, industry12, industry0, industry13. If industry6, industry1, industry3, industry11, __index_level_0__, industry7, industry9, question, Description, industry8, industry5, industry2, industry4, industry10, industry12, industry0, industry13 are not expected by `BertForMultipleChoice.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 38
  Batch size = 2


In [ ]:
logits = torch.tensor(output.predictions)
y_preds = torch.nn.Softmax(dim=1)(logits)
y_preds = torch.argmax(y_preds, dim=1)

In [ ]:
y_true = tokenized_i_data['test']['label']

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(y_true, y_preds)

0.8157894736842105

### Label Entire Dataset

In [ ]:
saved_model_file = '/content/drive/MyDrive/EDAV Final Project/Bert_MC_ids_0.76.pt'

In [ ]:
model.load_state_dict(torch.load(saved_model_file))

<All keys matched successfully>

In [ ]:
raw_data = pd.read_csv('cybersecurity_incidents_descriptions_v2.csv', encoding = "ISO-8859-1")
raw_data['question'] = current_question
for k, v in industries.items():
  raw_data[k] = v

raw_data.head()

,Unnamed: 0,Description,question,industry0,industry1,industry2,industry3,industry4,industry5,industry6,industry7,industry8,industry9,industry10,industry11,industry12,industry13
0,0,September 2022. Iranian hackers targeted Alban...,The industry targeted was,Unknown,Telecommunications,Defense,Education,Energy,Finance,Government,Healthcare,Manufacturing,Media,Non-profit,Technology,Transportation,Utilities
1,1,September 2022. Hackers targeted Montenegroâ...,The industry targeted was,Unknown,Telecommunications,Defense,Education,Energy,Finance,Government,Healthcare,Manufacturing,Media,Non-profit,Technology,Transportation,Utilities
2,2,September 2022. Hackers targeted the state -le...,The industry targeted was,Unknown,Telecommunications,Defense,Education,Energy,Finance,Government,Healthcare,Manufacturing,Media,Non-profit,Technology,Transportation,Utilities
3,3,September 2022. China accused the U.S. Nationa...,The industry targeted was,Unknown,Telecommunications,Defense,Education,Energy,Finance,Government,Healthcare,Manufacturing,Media,Non-profit,Technology,Transportation,Utilities
4,4,September 2022. The group Anonymous took respo...,The industry targeted was,Unknown,Telecommunications,Defense,Education,Energy,Finance,Government,Healthcare,Manufacturing,Media,Non-profit,Technology,Transportation,Utilities


In [ ]:
r_dataset = Dataset.from_pandas(raw_data)
tokenized_r_data = r_dataset.map(preprocess_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
@dataclass
class DataCollatorForMCPrediction:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        #label_name = "label" if "label" in features[0].keys() else "labels"
        #labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])

        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        #batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        return batch

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=30,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_r_data,
    eval_dataset=tokenized_r_data,
    tokenizer=tokenizer,
    data_collator=DataCollatorForMCPrediction(tokenizer=tokenizer),
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
with torch.no_grad():
  model.eval()
  all_labels = trainer.predict(tokenized_r_data)

The following columns in the test set don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: industry6, industry1, industry3, industry11, industry7, industry9, question, Description, industry8, industry5, industry2, industry4, industry10, Unnamed: 0, industry12, industry0, industry13. If industry6, industry1, industry3, industry11, industry7, industry9, question, Description, industry8, industry5, industry2, industry4, industry10, Unnamed: 0, industry12, industry0, industry13 are not expected by `BertForMultipleChoice.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 914
  Batch size = 2


In [ ]:
all_labels

PredictionOutput(predictions=array([[-8.192037 , -8.244708 , -8.1920395, ..., -8.166098 , -8.231419 ,
        -8.168997 ],
       [-8.112663 , -8.198075 , -8.140498 , ..., -7.993029 , -8.209862 ,
        -8.094009 ],
       [-8.13431  , -8.238535 , -8.173097 , ..., -8.092962 , -8.226582 ,
        -8.113072 ],
       ...,
       [-8.167274 , -8.239814 , -8.090993 , ..., -8.221426 , -8.25361  ,
        -8.199372 ],
       [-8.068818 , -8.253564 ,  8.1520405, ..., -8.139965 , -8.254884 ,
        -8.211867 ],
       [-8.130209 , -8.261604 ,  8.105889 , ..., -8.241683 , -8.256551 ,
        -8.203951 ]], dtype=float32), label_ids=None, metrics={'test_runtime': 48.2422, 'test_samples_per_second': 18.946, 'test_steps_per_second': 9.473})

In [ ]:
logits = torch.tensor(all_labels.predictions)
all_preds = torch.nn.Softmax(dim=1)(logits)
all_preds = torch.argmax(all_preds, dim=1)
all_preds

tensor([ 6,  6,  6,  3,  6,  2,  2,  4,  6,  6,  4,  6,  4, 13,  6,  6,  6,  6,
         6,  6,  2,  4,  6,  6,  1,  6,  1,  6,  8,  6,  9,  6,  4,  6,  2,  6,
         6,  1,  6,  4,  7,  8, 11,  6,  6,  6,  6,  5,  6,  8,  6,  5,  2,  2,
         6,  4,  1,  9,  1,  4,  6,  1,  6,  1,  6,  6,  0,  5,  4,  6,  6,  6,
         1,  9,  6,  6,  5,  6,  2,  6,  2,  4,  0,  4,  6,  6,  1,  1,  6,  1,
         6,  6, 10,  6,  9,  6,  6,  6,  5,  1,  1,  2,  4,  6,  2, 12,  1,  6,
         6,  0,  1,  4,  2,  6,  2, 11,  1,  6,  6,  0,  2,  6,  5, 10,  2,  0,
         6,  6,  6,  6,  6,  6,  7,  1,  6,  4,  6, 11, 12,  6, 11,  4,  2,  2,
         6, 12,  1,  2,  6,  6, 10,  1,  6,  6,  2,  6,  4,  2,  6,  6,  4,  4,
         0,  6,  6,  7,  8,  4,  4,  6,  2,  6,  2,  1,  6,  0,  2,  0,  2, 12,
         2,  5,  6,  9,  2,  7,  6,  9,  6,  6,  6,  0, 12,  7,  7,  6,  6,  6,
         6,  4,  7,  2,  2,  6,  6,  2,  5,  6,  7,  6,  1,  6,  1, 13,  0,  1,
         0,  2,  5,  0,  4,  0,  6,  2, 

In [ ]:
labeled_data = pd.DataFrame(raw_data['Description'])
labeled_data['industry_class'] = all_preds
labeled_data['industry'] = labeled_data['industry_class'].apply(lambda x: int_to_industries[x])

In [ ]:
labeled_data.head()

,Description,industry_class,industry
0,September 2022. Iranian hackers targeted Alban...,6,Government
1,September 2022. Hackers targeted Montenegroâ...,6,Government
2,September 2022. Hackers targeted the state -le...,6,Government
3,September 2022. China accused the U.S. Nationa...,3,Education
4,September 2022. The group Anonymous took respo...,6,Government


In [ ]:
labeled_data.to_csv('cyber_incidents_w_industry.csv')

### Compare to previously labeled train and test data

In [ ]:
data_ids = pd.read_csv('cyber_incidents_w_labels_v3.csv', index_col=0)
data_ids

,Unnamed: 0.1,Description,Attacker,Victim,Industry
765,765,"March 2013. The Syrian Electronic Army, a pro ...",172,0,9
86,86,January 2022. A Chinese hacking group breached...,36,66,1
46,46,April 2022. Iranian state tele vision claimed ...,0,81,6
726,726,September 2014. A false Occupy Central smartph...,36,76,0
898,898,"September 2007. Francis D elon, Secretary -Gen...",36,62,2
...,...,...,...,...,...
454,454,January 2019. France attributed a cyberattack ...,146,62,2
105,105,November 2021. Chinese officials claim a forei...,0,36,12
595,595,August 2017 . Researchers inform the Estonian ...,0,57,6
709,709,March 2015. An Iranian hacking group conducted...,81,0,2


In [ ]:
results = pd.concat([labeled_data, data_ids], axis=1)[['Description', 'industry_class', 'industry', 'Industry']]

In [ ]:
results.to_csv('cyber_incidents_w_industry.csv')

In [ ]:
# mislabeled data
results[results['industry_class'] != results['Industry']].dropna().count()

Description       6
Description       6
industry_class    6
industry          6
Industry          6
dtype: int64

In [ ]:
results[results['industry_class'] != results['Industry']].dropna()

,Description,Description,industry_class,industry,Industry
33,June 2022. Hackers targeted Harmonyâs Horizo...,June 2022. Hackers targeted Harmonyâs Horizo...,6,Government,5.0
202,February 2021 . Hackers associated with the Ch...,February 2021 . Hackers associated with the Ch...,2,Defense,0.0
726,September 2014. A false Occupy Central smartph...,September 2014. A false Occupy Central smartph...,6,Government,0.0
734,May 2014. Alleged Chinese hackers posed as C -...,May 2014. Alleged Chinese hackers posed as C -...,11,Technology,8.0
893,"November 2007. Jonathan Evans, the head of Bri...","November 2007. Jonathan Evans, the head of Bri...",2,Defense,0.0
894,October 2007. More than a thousand staffers at...,October 2007. More than a thousand staffers at...,11,Technology,2.0


### Compare Unseen Data

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
labeled_ids = pd.read_csv('cyber_incidents_w_industry.csv', index_col=0)
labeled_ids.head()

,Description,Description.1,industry_class,industry,Industry
0,September 2022. Iranian hackers targeted Alban...,September 2022. Iranian hackers targeted Alban...,6,Government,6.0
1,September 2022. Hackers targeted Montenegroâ...,NaN,6,Government,NaN
2,September 2022. Hackers targeted the state -le...,September 2022. Hackers targeted the state -le...,6,Government,6.0
3,September 2022. China accused the U.S. Nationa...,September 2022. China accused the U.S. Nationa...,3,Education,3.0
4,September 2022. The group Anonymous took respo...,NaN,6,Government,NaN


In [ ]:
unseen_data = pd.read_csv('cyber_incidents_w_labels_unseen.csv', index_col=0)
unseen_data.head()

,Unnamed: 0.1,Description,hacker_label,victim_label,industry_label
830,830,December 2010. British Foreign Minister Willia...,0,187,6
220,220,"December 2020. On Christmas Eve, hackers hit t...",0,187,6
106,106,November 2021 . After CISA publicly shared det...,36,0,0
47,47,April 2022 . Russian hackers targeted the Cost...,146,41,6
282,282,September 2020. Norway announced it had defend...,146,133,6


In [ ]:
ids_data = pd.concat([labeled_ids, unseen_data], axis=1)
ids_data.head()

,Description,Description.1,industry_class,industry,Industry,Unnamed: 0.1,Description,hacker_label,victim_label,industry_label
0,September 2022. Iranian hackers targeted Alban...,September 2022. Iranian hackers targeted Alban...,6,Government,6.0,NaN,NaN,NaN,NaN,NaN
1,September 2022. Hackers targeted Montenegroâ...,NaN,6,Government,NaN,NaN,NaN,NaN,NaN,NaN
2,September 2022. Hackers targeted the state -le...,September 2022. Hackers targeted the state -le...,6,Government,6.0,NaN,NaN,NaN,NaN,NaN
3,September 2022. China accused the U.S. Nationa...,September 2022. China accused the U.S. Nationa...,3,Education,3.0,NaN,NaN,NaN,NaN,NaN
4,September 2022. The group Anonymous took respo...,NaN,6,Government,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# check accuracy
ids_data[ids_data['industry_class'] != ids_data['industry_label']][['Description', 'industry_class', 'industry_label']].dropna().count()

Description       13
Description       13
industry_class    13
industry_label    13
dtype: int64

In [ ]:
ids_data[ids_data['industry_class'] != ids_data['industry_label']][['Description', 'industry_class', 'industry_label']].dropna()

,Description,Description,industry_class,industry_label
47,April 2022 . Russian hackers targeted the Cost...,April 2022 . Russian hackers targeted the Cost...,5,6.0
58,April 2022 . The United States removed Russian...,April 2022 . The United States removed Russian...,1,0.0
106,November 2021 . After CISA publicly shared det...,November 2021 . After CISA publicly shared det...,11,0.0
220,"December 2020. On Christmas Eve, hackers hit t...","December 2020. On Christmas Eve, hackers hit t...",4,6.0
300,July 2020 . President Trump confirmed that he ...,July 2020 . President Trump confirmed that he ...,11,6.0
527,June 2018. Ukraine police claim that Russian h...,June 2018. Ukraine police claim that Russian h...,4,5.0
596,August 2017. South Koreaâs Cyber Warfare Res...,August 2017. South Koreaâs Cyber Warfare Res...,6,5.0
608,June 2017. The New York Times revealed that sp...,June 2017. The New York Times revealed that sp...,6,10.0
748,August 2013. The Syrian Electronic Army hijack...,August 2013. The Syrian Electronic Army hijack...,11,9.0
801,March 2012. Trend Micro uncovered a Chinese cy...,March 2012. Trend Micro uncovered a Chinese cy...,0,2.0


In [ ]:
ids_data.to_csv('industry_w_unseen_labels.csv')

# Multiple Choice NLP - Type of Hack

In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 32.6 MB/s 
     |████████████████████████████████| 132 kB 73.6 MB/s 
     |████████████████████████████████| 212 kB 78.7 MB/s 
     |████████████████████████████████| 127 kB 69.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
hacktype_list = np.load('hacktype_list.npy')
hacktypes = {}
int_to_hacktype = {}

for (i, ht) in enumerate(hacktype_list):
  hacktypes[f'ht{i}'] = ht
  int_to_hacktype[i] = ht

In [ ]:
data_w_labels = pd.read_csv('cyber_incidents_w_labels_v3.csv', encoding="ISO-8859-1", index_col=0)
data_w_labels.head()

,Unnamed: 0.1,Description,Attacker,Victim,Industry,Hacktype
765,765,"March 2013. The Syrian Electronic Army, a pro ...",172,0,9,0
86,86,January 2022. A Chinese hacking group breached...,36,66,1,2
46,46,April 2022. Iranian state tele vision claimed ...,0,81,6,0
726,726,September 2014. A false Occupy Central smartph...,36,76,0,4
898,898,"September 2007. Francis D elon, Secretary -Gen...",36,62,2,0


In [ ]:
extra_data = pd.read_csv('cyber_incidents_w_labels_extra_train.csv', encoding="ISO-8859-1", index_col=0)
extra_data.rename(columns={'hacktype_label': 'Hacktype'}, inplace=True)
extra_data.head()

,Unnamed: 0.1,Description,Hacktype
823,823,"April 2011. Between March 2010 and April 2011,...",0
218,218,January 2021. Unidentified hackers breached on...,2
111,111,October 2021. A cyberattack targeted the gover...,0
48,48,April 2022. Hackers targeted members of the Eu...,4
283,283,August 2020 . A North Korean hacking group tar...,5


In [ ]:
data_w_labels = pd.concat([data_w_labels, extra_data])
data_w_labels

,Unnamed: 0.1,Description,Attacker,Victim,Industry,Hacktype
765,765,"March 2013. The Syrian Electronic Army, a pro ...",172.0,0.0,9.0,0
86,86,January 2022. A Chinese hacking group breached...,36.0,66.0,1.0,2
46,46,April 2022. Iranian state tele vision claimed ...,0.0,81.0,6.0,0
726,726,September 2014. A false Occupy Central smartph...,36.0,76.0,0.0,4
898,898,"September 2007. Francis D elon, Secretary -Gen...",36.0,62.0,2.0,0
...,...,...,...,...,...,...
25,25,June 2022. Hackers targeted LithuaniaÃ¢ÂÂs s...,NaN,NaN,NaN,1
739,739,March 2014. Cybercriminals steal 40 million cr...,NaN,NaN,NaN,2
635,635,February 2017. Hackers compromised the Singapo...,NaN,NaN,NaN,2
302,302,June 2020 . A hacking group affiliated with an...,NaN,NaN,NaN,0


In [ ]:
current_question = 'The type of hack that occurred was'

In [ ]:
data_ht = data_w_labels[['Description', 'Hacktype']]
data_ht['question'] = current_question
data_ht.rename(columns={'Hacktype': 'label'}, inplace=True)

for k, v in hacktypes.items():
  data_ht[k] = v

data_ht.head()

<ipython-input-63-9b9f7b45a350>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_ht['question'] = current_question
/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
<ipython-input-63-9b9f7b45a350>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

,Description,label,question,ht0,ht1,ht2,ht3,ht4,ht5,ht6,ht7
765,"March 2013. The Syrian Electronic Army, a pro ...",0,The type of hack that occurred was,Unknown,Denial of Service,Data Exfiltration,Misinformation,Malware,Phishing,Zero-day Exploits,Supply Chain
86,January 2022. A Chinese hacking group breached...,2,The type of hack that occurred was,Unknown,Denial of Service,Data Exfiltration,Misinformation,Malware,Phishing,Zero-day Exploits,Supply Chain
46,April 2022. Iranian state tele vision claimed ...,0,The type of hack that occurred was,Unknown,Denial of Service,Data Exfiltration,Misinformation,Malware,Phishing,Zero-day Exploits,Supply Chain
726,September 2014. A false Occupy Central smartph...,4,The type of hack that occurred was,Unknown,Denial of Service,Data Exfiltration,Misinformation,Malware,Phishing,Zero-day Exploits,Supply Chain
898,"September 2007. Francis D elon, Secretary -Gen...",0,The type of hack that occurred was,Unknown,Denial of Service,Data Exfiltration,Misinformation,Malware,Phishing,Zero-day Exploits,Supply Chain


In [ ]:
from datasets import Dataset, DatasetDict

In [ ]:
data_ht = data_w_labels[['Description', 'Hacktype']]
data_ht.rename(columns={'Hacktype': 'label'}, inplace=True)
data_ht.rename(columns={'Description': 'text'}, inplace=True)

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [ ]:
data_ht.head()

,Description,label,question,ht0,ht1,ht2,ht3,ht4,ht5,ht6,ht7
765,"March 2013. The Syrian Electronic Army, a pro ...",0,The type of hack that occurred was,Unknown,Denial of Service,Data Exfiltration,Misinformation,Malware,Phishing,Zero-day Exploits,Supply Chain
86,January 2022. A Chinese hacking group breached...,2,The type of hack that occurred was,Unknown,Denial of Service,Data Exfiltration,Misinformation,Malware,Phishing,Zero-day Exploits,Supply Chain
46,April 2022. Iranian state tele vision claimed ...,0,The type of hack that occurred was,Unknown,Denial of Service,Data Exfiltration,Misinformation,Malware,Phishing,Zero-day Exploits,Supply Chain
726,September 2014. A false Occupy Central smartph...,4,The type of hack that occurred was,Unknown,Denial of Service,Data Exfiltration,Misinformation,Malware,Phishing,Zero-day Exploits,Supply Chain
898,"September 2007. Francis D elon, Secretary -Gen...",0,The type of hack that occurred was,Unknown,Denial of Service,Data Exfiltration,Misinformation,Malware,Phishing,Zero-day Exploits,Supply Chain


In [ ]:
ht_dataset = Dataset.from_pandas(data_ht)
#ht_dataset

In [ ]:
test_split = 0.25

train_dataset, test_dataset = ht_dataset.train_test_split(test_size=test_split).values()

In [ ]:
h_dataset = DatasetDict({'train': train_dataset,
                         'test': test_dataset})
#h_dataset

### Create DataCollator


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

KeyboardInterrupt: ignored

In [ ]:
num_hacktypes = len(hacktypes)
hacktype_keys = hacktypes.keys()

def preprocess_function(examples):
    first_sentences = [[context] * num_hacktypes for context in examples["Description"]]
    #print(first_sentences)
    question_headers = examples["question"]
    second_sentences = [
        [f"{header} {examples[hacktype][i]}" for hacktype in hacktype_keys] for i, header in enumerate(question_headers)
    ]

    first_sentences = sum(first_sentences, [])
    second_sentences = sum(second_sentences, [])

    # print(first_sentences)
    # print(second_sentences)

    tokenized_examples = tokenizer(first_sentences, second_sentences, truncation=True)
    return {k: [v[i : i + num_hacktypes] for i in range(0, len(v), num_hacktypes)] for k, v in tokenized_examples.items()}

In [ ]:
tokenized_h_data = h_dataset.map(preprocess_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import torch

In [ ]:
@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])

        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        return batch

### Train Model - Industry Hacked


In [ ]:
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer

model = AutoModelForMultipleChoice.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMultipleChoice: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly

Load saved weights (if retraining)

In [ ]:
saved_model_file = '/content/drive/MyDrive/EDAV Final Project/Bert_MC_ht_.pt'

In [ ]:
model.load_state_dict(torch.load(saved_model_file))

<All keys matched successfully>

In [ ]:
import torch

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
model.to(device)

BertForMultipleChoice(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31820, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_aff

In [ ]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 1.5 MB/s 


In [ ]:
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
# freeze model weights except classification layer and last BERT layer
for param in model.parameters():
  param.requires_grad = False

for param in model.classifier.parameters():
  param.requires_grad = True

for param in model.bert.encoder.layer[11:].parameters():
  param.requires_grad = True

In [ ]:
for param in model.parameters():
  param.requires_grad = True

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-7,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_h_data["train"],
    eval_dataset=tokenized_h_data["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator=DataCollatorForMultipleChoice(tokenizer=tokenizer),
)

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: ht1, ht0, question, ht7, ht5, ht4, ht3, Description, ht2, __index_level_0__, ht6. If ht1, ht0, question, ht7, ht5, ht4, ht3, Description, ht2, __index_level_0__, ht6 are not expected by `BertForMultipleChoice.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
****

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,3.135745,0.700000
2,No log,3.146376,0.700000


The following columns in the evaluation set don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: ht1, ht0, question, ht7, ht5, ht4, ht3, Description, ht2, __index_level_0__, ht6. If ht1, ht0, question, ht7, ht5, ht4, ht3, Description, ht2, __index_level_0__, ht6 are not expected by `BertForMultipleChoice.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 50
  Batch size = 2
The following columns in the evaluation set don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: ht1, ht0, question, ht7, ht5, ht4, ht3, Description, ht2, __index_level_0__, ht6. If ht1, ht0, question, ht7, ht5, ht4, ht3, Description, ht2, __index_level_0__, ht6 are not expected by `BertForMultipleChoice.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 50
  Batch size = 2


KeyboardInterrupt: ignored

In [ ]:
# save model weights
torch.save(model.state_dict(), 'Bert_MC_hacktype_0.80.pt')

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp Bert_MC_hacktype_0.80.pt /content/drive/MyDrive/EDAV\ Final\ Project/

### Validate Accuracy on validation set

In [ ]:
tokenized_h_data['test']

Dataset({
    features: ['Description', 'label', 'question', 'ht0', 'ht1', 'ht2', 'ht3', 'ht4', 'ht5', 'ht6', 'ht7', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 50
})

In [ ]:
output = trainer.predict(tokenized_h_data['test'])

The following columns in the test set don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: ht1, ht0, question, ht7, ht5, ht4, ht3, Description, ht2, __index_level_0__, ht6. If ht1, ht0, question, ht7, ht5, ht4, ht3, Description, ht2, __index_level_0__, ht6 are not expected by `BertForMultipleChoice.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 50
  Batch size = 2


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,3.135745,0.700000
2,No log,3.146376,0.700000


In [ ]:
logits = torch.tensor(output.predictions)
y_preds = torch.nn.Softmax(dim=1)(logits)
y_preds = torch.argmax(y_preds, dim=1)

In [ ]:
y_true = tokenized_h_data['test']['label']

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(y_true, y_preds)

0.7

### Label Entire Dataset

In [ ]:
saved_model_file = '/content/drive/MyDrive/EDAV Final Project/Bert_MC_hacktype_0.68.pt'

In [ ]:
saved_model_file = 'Bert_MC_hacktype_0.80.pt'

In [ ]:
model.load_state_dict(torch.load(saved_model_file))

<All keys matched successfully>

In [ ]:
raw_data = pd.read_csv('cybersecurity_incidents_descriptions_v2.csv', encoding = "ISO-8859-1")
raw_data['question'] = current_question
for k, v in hacktypes.items():
  raw_data[k] = v

raw_data.head()

,Unnamed: 0,Description,question,ht0,ht1,ht2,ht3,ht4,ht5,ht6,ht7
0,0,September 2022. Iranian hackers targeted Alban...,The type of hack that occurred was,Unknown,Denial of Service,Data Exfiltration,Misinformation,Malware,Phishing,Zero-day Exploits,Supply Chain
1,1,September 2022. Hackers targeted Montenegroâ...,The type of hack that occurred was,Unknown,Denial of Service,Data Exfiltration,Misinformation,Malware,Phishing,Zero-day Exploits,Supply Chain
2,2,September 2022. Hackers targeted the state -le...,The type of hack that occurred was,Unknown,Denial of Service,Data Exfiltration,Misinformation,Malware,Phishing,Zero-day Exploits,Supply Chain
3,3,September 2022. China accused the U.S. Nationa...,The type of hack that occurred was,Unknown,Denial of Service,Data Exfiltration,Misinformation,Malware,Phishing,Zero-day Exploits,Supply Chain
4,4,September 2022. The group Anonymous took respo...,The type of hack that occurred was,Unknown,Denial of Service,Data Exfiltration,Misinformation,Malware,Phishing,Zero-day Exploits,Supply Chain


In [ ]:
r_dataset = Dataset.from_pandas(raw_data)
tokenized_r_data = r_dataset.map(preprocess_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
raw_data.rename(columns={'Description': 'text'}, inplace=True)
r_dataset = Dataset.from_pandas(raw_data)
tokenized_r_data = r_dataset.map(tokenize_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
@dataclass
class DataCollatorForMCPrediction:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        #label_name = "label" if "label" in features[0].keys() else "labels"
        #labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])

        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        #batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        return batch

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=30,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_r_data,
    eval_dataset=tokenized_r_data,
    tokenizer=tokenizer,
    data_collator=DataCollatorForMCPrediction(tokenizer=tokenizer),
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
with torch.no_grad():
  model.eval()
  all_labels = trainer.predict(tokenized_r_data)

The following columns in the test set don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: ht1, ht0, question, ht7, ht5, ht4, Unnamed: 0, ht3, Description, ht2, ht6. If ht1, ht0, question, ht7, ht5, ht4, Unnamed: 0, ht3, Description, ht2, ht6 are not expected by `BertForMultipleChoice.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 914
  Batch size = 2


In [ ]:
all_labels

PredictionOutput(predictions=array([[-10.366316  , -10.586734  , -10.5003    , ..., -10.957391  ,
        -11.069314  ,  -6.275503  ],
       [  0.43986842,  -4.3479943 , -10.627816  , ..., -10.759397  ,
        -10.99795   ,  -6.45891   ],
       [ -9.769604  ,  10.31362   , -10.547094  , ..., -10.85837   ,
        -11.02408   ,  -7.017061  ],
       ...,
       [ -8.966677  , -10.441215  , -10.161492  , ..., -10.812185  ,
        -11.003706  ,  -6.130052  ],
       [  7.6853805 , -10.6477375 , -10.505433  , ..., -10.750922  ,
        -11.003121  ,  -6.7840033 ],
       [ -9.58395   , -10.661449  ,  10.830752  , ..., -10.756384  ,
        -11.038266  ,  -6.623842  ]], dtype=float32), label_ids=None, metrics={'test_runtime': 33.247, 'test_samples_per_second': 27.491, 'test_steps_per_second': 13.746})

In [ ]:
logits = torch.tensor(all_labels.predictions)
all_preds = torch.nn.Softmax(dim=1)(logits)
all_preds = torch.argmax(all_preds, dim=1)
all_preds

tensor([4, 0, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 2, 2, 0, 1, 4, 2, 2, 0, 1, 1, 1, 0,
        2, 1, 7, 5, 0, 0, 7, 0, 3, 2, 5, 0, 0, 1, 5, 0, 1, 2, 1, 0, 0, 4, 0, 2,
        4, 5, 4, 2, 1, 4, 5, 0, 5, 2, 4, 4, 7, 1, 2, 7, 2, 1, 0, 6, 4, 1, 0, 6,
        1, 4, 1, 2, 3, 2, 2, 2, 2, 4, 4, 7, 2, 0, 2, 0, 1, 1, 0, 1, 2, 4, 0, 4,
        7, 4, 7, 7, 0, 2, 4, 4, 0, 4, 0, 2, 0, 2, 2, 7, 0, 4, 2, 0, 2, 0, 0, 2,
        7, 0, 7, 2, 2, 0, 7, 0, 2, 0, 7, 0, 7, 0, 2, 2, 0, 0, 0, 0, 0, 0, 4, 1,
        0, 0, 4, 0, 0, 2, 3, 0, 7, 0, 2, 0, 0, 0, 4, 0, 4, 7, 4, 2, 2, 4, 4, 4,
        4, 0, 0, 7, 2, 4, 5, 2, 7, 5, 0, 0, 0, 0, 2, 2, 3, 2, 0, 4, 0, 0, 0, 0,
        2, 1, 2, 2, 0, 2, 2, 0, 2, 2, 0, 4, 2, 1, 2, 7, 7, 2, 0, 4, 5, 0, 2, 2,
        3, 0, 2, 4, 2, 5, 0, 2, 7, 0, 2, 0, 4, 2, 2, 2, 4, 0, 5, 4, 4, 7, 2, 7,
        0, 2, 3, 0, 0, 0, 0, 0, 2, 0, 1, 2, 0, 0, 5, 2, 2, 0, 4, 0, 0, 2, 0, 4,
        0, 6, 2, 2, 1, 0, 4, 4, 4, 2, 4, 0, 4, 0, 0, 2, 4, 0, 0, 5, 2, 1, 2, 4,
        5, 2, 0, 7, 0, 2, 3, 0, 2, 2, 2,

In [ ]:
labeled_data = pd.DataFrame(raw_data['Description'])
labeled_data['hacktype_class'] = all_preds
labeled_data['hacktype'] = labeled_data['hacktype_class'].apply(lambda x: int_to_hacktype[x])

In [ ]:
labeled_data.head()

,Description,hacktype_class,hacktype
0,September 2022. Iranian hackers targeted Alban...,4,Malware
1,September 2022. Hackers targeted Montenegroâ...,0,Unknown
2,September 2022. Hackers targeted the state -le...,1,Denial of Service
3,September 2022. China accused the U.S. Nationa...,2,Data Exfiltration
4,September 2022. The group Anonymous took respo...,1,Denial of Service


In [ ]:
labeled_data.to_csv('cyber_incidents_w_ht.csv')

### Compare to previously labeled train and test data

In [ ]:
data_ht = pd.read_csv('cyber_incidents_w_labels_v3.csv', index_col=0)
data_ht

,Unnamed: 0.1,Description,Attacker,Victim,Industry,Hacktype
765,765,"March 2013. The Syrian Electronic Army, a pro ...",172,0,9,0
86,86,January 2022. A Chinese hacking group breached...,36,66,1,2
46,46,April 2022. Iranian state tele vision claimed ...,0,81,6,0
726,726,September 2014. A false Occupy Central smartph...,36,76,0,4
898,898,"September 2007. Francis D elon, Secretary -Gen...",36,62,2,0
...,...,...,...,...,...,...
454,454,January 2019. France attributed a cyberattack ...,146,62,2,0
105,105,November 2021. Chinese officials claim a forei...,0,36,12,4
595,595,August 2017 . Researchers inform the Estonian ...,0,57,6,2
709,709,March 2015. An Iranian hacking group conducted...,81,0,2,5


In [ ]:
results = pd.concat([labeled_data, data_ht], axis=1)[['Description', 'hacktype_class', 'hacktype', 'Hacktype']]

In [ ]:
results.to_csv('cyber_incidents_w_hacktype.csv')

In [ ]:
# mislabeled data
results[results['hacktype_class'] != results['Hacktype']].dropna().count()

Description       16
Description       16
hacktype_class    16
hacktype          16
Hacktype          16
dtype: int64

In [ ]:
results[results['hacktype_class'] != results['Hacktype']].dropna()

,Description,Description,industry_class,industry,Industry
33,June 2022. Hackers targeted Harmonyâs Horizo...,June 2022. Hackers targeted Harmonyâs Horizo...,6,Government,5.0
202,February 2021 . Hackers associated with the Ch...,February 2021 . Hackers associated with the Ch...,2,Defense,0.0
726,September 2014. A false Occupy Central smartph...,September 2014. A false Occupy Central smartph...,6,Government,0.0
734,May 2014. Alleged Chinese hackers posed as C -...,May 2014. Alleged Chinese hackers posed as C -...,11,Technology,8.0
893,"November 2007. Jonathan Evans, the head of Bri...","November 2007. Jonathan Evans, the head of Bri...",2,Defense,0.0
894,October 2007. More than a thousand staffers at...,October 2007. More than a thousand staffers at...,11,Technology,2.0


### Compare Unseen Data

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
labeled_hacktype = pd.read_csv('cyber_incidents_w_hacktype.csv', index_col=0)
labeled_hacktype.head()

FileNotFoundError: ignored

In [ ]:
unseen_data = pd.read_csv('cyber_incidents_w_labels_unseen.csv', index_col=0)
unseen_data.head()

,Unnamed: 0.1,Description,hacker_label,victim_label,industry_label,hacktype_label
830,830,December 2010. British Foreign Minister Willia...,0,187,6,0
220,220,"December 2020. On Christmas Eve, hackers hit t...",0,187,6,4
106,106,November 2021 . After CISA publicly shared det...,36,0,0,6
47,47,April 2022 . Russian hackers targeted the Cost...,146,41,6,2
282,282,September 2020. Norway announced it had defend...,146,133,6,0


In [ ]:
ht_data = pd.concat([labeled_data, unseen_data], axis=1)
ht_data.head()

,Description,hacktype_class,hacktype,Unnamed: 0.1,Description,hacker_label,victim_label,industry_label,hacktype_label
0,September 2022. Iranian hackers targeted Alban...,4,Malware,NaN,NaN,NaN,NaN,NaN,NaN
1,September 2022. Hackers targeted Montenegroâ...,0,Unknown,NaN,NaN,NaN,NaN,NaN,NaN
2,September 2022. Hackers targeted the state -le...,1,Denial of Service,NaN,NaN,NaN,NaN,NaN,NaN
3,September 2022. China accused the U.S. Nationa...,2,Data Exfiltration,NaN,NaN,NaN,NaN,NaN,NaN
4,September 2022. The group Anonymous took respo...,1,Denial of Service,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# check accuracy
ht_data[ht_data['hacktype_class'] != ht_data['hacktype_label']][['Description', 'hacktype_class', 'hacktype_label']].dropna().count()

Description       14
Description       14
hacktype_class    14
hacktype_label    14
dtype: int64

In [ ]:
ht_data[ht_data['hacktype_class'] != ht_data['hacktype_label']][['Description', 'hacktype_class', 'hacktype_label']].dropna()

,Description,Description,hacktype_class,hacktype_label
171,April 2021 . A hacking group compromised the s...,April 2021 . A hacking group compromised the s...,0,3.0
231,December 2020 . Over 200 organizations around ...,December 2020 . Over 200 organizations around ...,2,7.0
493,October 2018. News reports reveal that the Isr...,October 2018. News reports reveal that the Isr...,4,2.0
641,November 2016. An indiscriminate attack compro...,November 2016. An indiscriminate attack compro...,0,1.0
654,July 2016. Forensic evidence points to Russian...,July 2016. Forensic evidence points to Russian...,0,2.0
748,August 2013. The Syrian Electronic Army hijack...,August 2013. The Syrian Electronic Army hijack...,1,5.0
801,March 2012. Trend Micro uncovered a Chinese cy...,March 2012. Trend Micro uncovered a Chinese cy...,4,0.0
863,March 2009. Reports in the press say that the ...,March 2009. Reports in the press say that the ...,0,2.0
897,September 2007. Contractors employed by DHS an...,September 2007. Contractors employed by DHS an...,0,7.0
910,May 2006. The Department of Stateâs networks...,May 2006. The Department of Stateâs networks...,0,2.0


In [ ]:
ht_data.to_csv('ht_w_unseen_labels.csv')

# Combine Results 

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime

## Date

In [ ]:
date_data = pd.read_csv('/content/cybersecurity_incidents_with_dates.csv', index_col=0)
date_data['time-mY'] = pd.to_datetime(date_data['time_clean'], format='%b-%y', errors='coerce')
date_data['time-Y'] = pd.to_datetime(date_data['time_clean'], format='%Y', errors='coerce')

In [ ]:
date_labels = pd.DataFrame(date_data['time-mY'].fillna(date_data['time-Y'][date_data['time-Y'].notnull()]))
date_labels.head()

,time-mY
0,2022-09-01
1,2022-09-01
2,2022-09-01
3,2022-09-01
4,2022-09-01


In [ ]:
date_labels['time-mY'].isna().sum()

0

In [ ]:
idx_to_mth = {
    1: 'January',
    2: 'February',
    3: 'March',
    4: 'April',
    5: 'May',
    6: 'June',
    7: 'July',
    8: 'August',
    9: 'September',
    10: 'October',
    11: 'November',
    12: 'December'
}

In [ ]:
# get month and year
date_labels['month'] = date_labels['time-mY'].apply(lambda x: idx_to_mth[x.month])
date_labels['year'] = date_labels['time-mY'].apply(lambda x: x.year)
date_labels.head()

,time-mY,month,year
0,2022-09-01,September,2022
1,2022-09-01,September,2022
2,2022-09-01,September,2022
3,2022-09-01,September,2022
4,2022-09-01,September,2022


In [ ]:
# set labels with unknown months
date_labels.loc[date_data['time-Y'].notnull(), 'month'] = 'Unknown'

## Hacker Origin

In [ ]:
hacker_data = pd.read_csv('/content/hacker_origin_w_unseen_labels.csv', index_col=0)
hacker_data.head()

,Description,Description.1,hacker_origin_class,hacker_origin,Attacker,Unnamed: 0.1,Description.1.1,hacker_label,victim_label
0,September 2022. Iranian hackers targeted Alban...,September 2022. Iranian hackers targeted Alban...,81,Iran,81.0,NaN,NaN,NaN,NaN
1,September 2022. Hackers targeted Montenegroâ...,NaN,146,Russia,NaN,NaN,NaN,NaN,NaN
2,September 2022. Hackers targeted the state -le...,September 2022. Hackers targeted the state -le...,0,Unknown,0.0,NaN,NaN,NaN,NaN
3,September 2022. China accused the U.S. Nationa...,September 2022. China accused the U.S. Nationa...,36,China,188.0,NaN,NaN,NaN,NaN
4,September 2022. The group Anonymous took respo...,NaN,0,Unknown,NaN,NaN,NaN,NaN,NaN


In [ ]:
hacker_origin = hacker_data.loc[:, ['hacker_origin_class']].copy()
hacker_origin.head()

,hacker_origin_class
0,81
1,146
2,0
3,36
4,0


In [ ]:
hacker_data[hacker_data['hacker_origin_class'] != hacker_data['Attacker']][['hacker_origin_class', 'Attacker']].dropna()

,hacker_origin_class,Attacker
3,36,188.0
75,36,188.0
77,36,188.0
95,146,92.0
119,81,0.0
246,36,188.0
694,0,32.0
761,188,0.0


In [ ]:
# update values to labeled datah
hacker_origin.loc[hacker_data['Attacker'].notnull()] = hacker_data[['Attacker']].dropna()
hacker_origin.loc[hacker_data['hacker_label'].notnull()] = hacker_data[['hacker_label']].dropna()
# hacker_origin

In [ ]:
(hacker_origin.loc[hacker_data['Attacker'].notnull()]['hacker_origin_class'] != hacker_data['Attacker'].dropna()).sum()

0

In [ ]:
(hacker_origin.loc[hacker_data['hacker_label'].notnull()]['hacker_origin_class'] != hacker_data['hacker_label'].dropna()).sum()

0

## Victim Origin

In [ ]:
victim_data = pd.read_csv('/content/victim_origin_w_unseen_labels.csv', index_col=0)
victim_data.head()

,Description,Description.1,victim_origin_class,victim_origin,Victim,Unnamed: 0.1,Description.1.1,hacker_label,victim_label
0,September 2022. Iranian hackers targeted Alban...,September 2022. Iranian hackers targeted Alban...,2,Albania,2.0,NaN,NaN,NaN,NaN
1,September 2022. Hackers targeted Montenegroâ...,NaN,120,Montenegro,NaN,NaN,NaN,NaN,NaN
2,September 2022. Hackers targeted the state -le...,September 2022. Hackers targeted the state -le...,22,Bosnia and Herzegovina,22.0,NaN,NaN,NaN,NaN
3,September 2022. China accused the U.S. Nationa...,September 2022. China accused the U.S. Nationa...,36,China,36.0,NaN,NaN,NaN,NaN
4,September 2022. The group Anonymous took respo...,NaN,81,Iran,NaN,NaN,NaN,NaN,NaN


In [ ]:
victim_origin = victim_data.loc[:, ['victim_origin_class']].copy()
victim_origin.head()

,victim_origin_class
0,2
1,120
2,22
3,36
4,81


In [ ]:
# train and test labels
victim_data[victim_data['victim_origin_class'] != victim_data['Victim']][['victim_origin_class', 'Victim']].dropna()

,victim_origin_class,Victim
189,188,37.0
275,81,188.0
420,84,188.0
465,62,0.0
694,32,0.0
716,188,87.0
807,36,0.0
834,146,188.0


In [ ]:
# unseen labels
victim_data[victim_data['victim_origin_class'] != victim_data['victim_label']][['victim_origin_class', 'victim_label']].dropna()

,victim_origin_class,victim_label
26,0,188.0
654,146,188.0
748,146,0.0
802,36,188.0
863,81,188.0
878,188,0.0


In [ ]:
# update values to labeled data
victim_origin.loc[victim_data['Victim'].notnull()] = victim_data[['Victim']].dropna()   # train and test
victim_origin.loc[victim_data['victim_label'].notnull()] = victim_data[['victim_label']].dropna() # unseen
# victim_origin

In [ ]:
(victim_origin.loc[victim_data['Victim'].notnull()]['victim_origin_class'] != victim_data['Victim'].dropna()).sum()

0

In [ ]:
(victim_origin.loc[victim_data['victim_label'].notnull()]['victim_origin_class'] != victim_data['victim_label'].dropna()).sum()

0

### Get country names from class

In [ ]:
countries_list = np.load('countries_list.npy')

int_to_countries = {}

for i, c in enumerate(countries_list):
  int_to_countries[i] = c

In [ ]:
hacker_origin['country'] = hacker_origin['hacker_origin_class'].apply(lambda x: int_to_countries[x])
hacker_origin.head()

,hacker_origin_class,country
0,81.0,Iran
1,146.0,Russia
2,0.0,Unknown
3,188.0,United States
4,0.0,Unknown


In [ ]:
victim_origin['country'] = victim_origin['victim_origin_class'].apply(lambda x: int_to_countries[x])
victim_origin.head()

,victim_origin_class,country
0,2.0,Albania
1,120.0,Montenegro
2,22.0,Bosnia and Herzegovina
3,36.0,China
4,81.0,Iran


## Industry

In [ ]:
ids_data = pd.read_csv('/content/industry_w_unseen_labels.csv', index_col=0)
ids_data.head()

,Description,Description.1,industry_class,industry,Industry,Unnamed: 0.1,Description.1.1,hacker_label,victim_label,industry_label
0,September 2022. Iranian hackers targeted Alban...,September 2022. Iranian hackers targeted Alban...,6,Government,6.0,NaN,NaN,NaN,NaN,NaN
1,September 2022. Hackers targeted Montenegroâ...,NaN,6,Government,NaN,NaN,NaN,NaN,NaN,NaN
2,September 2022. Hackers targeted the state -le...,September 2022. Hackers targeted the state -le...,6,Government,6.0,NaN,NaN,NaN,NaN,NaN
3,September 2022. China accused the U.S. Nationa...,September 2022. China accused the U.S. Nationa...,3,Education,3.0,NaN,NaN,NaN,NaN,NaN
4,September 2022. The group Anonymous took respo...,NaN,6,Government,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
industry = ids_data.loc[:, ['industry_class']].copy()
industry.head()

,industry_class
0,6
1,6
2,6
3,3
4,6


In [ ]:
# train and test labels
ids_data[ids_data['industry_class'] != ids_data['Industry']][['industry_class', 'Industry']].dropna()

,industry_class,Industry
33,6,5.0
202,2,0.0
726,6,0.0
734,11,8.0
893,2,0.0
894,11,2.0


In [ ]:
# unseen labels
ids_data[ids_data['industry_class'] != ids_data['industry_label']][['industry_class', 'industry_label']].dropna()

,industry_class,industry_label
47,5,6.0
58,1,0.0
106,11,0.0
220,4,6.0
300,11,6.0
527,4,5.0
596,6,5.0
608,6,10.0
748,11,9.0
801,0,2.0


In [ ]:
# update values to labeled data
industry.loc[ids_data['Industry'].notnull()] = ids_data[['Industry']].dropna()   # train and test
industry.loc[ids_data['industry_label'].notnull()] = ids_data[['industry_label']].dropna() # unseen
# industry

In [ ]:
(industry.loc[ids_data['Industry'].notnull()]['industry_class'] != ids_data['Industry'].dropna()).sum()

0

In [ ]:
(industry.loc[ids_data['industry_label'].notnull()]['industry_class'] != ids_data['industry_label'].dropna()).sum()

0

### Get industry name from class

In [ ]:
ids_list = np.load('industries_list.npy')

int_to_ids = {}

for i, ids in enumerate(ids_list):
  int_to_ids[i] = ids

In [ ]:
industry['industry'] = industry['industry_class'].apply(lambda x: int_to_ids[x])
industry.head()

,industry_class,industry
0,6.0,Government
1,6.0,Government
2,6.0,Government
3,3.0,Education
4,6.0,Government


## Type of Hack

In [ ]:
ht_train = pd.read_csv('/content/cyber_incidents_w_labels_v3.csv', index_col=0)
ht_train.head()

,Unnamed: 0.1,Description,Attacker,Victim,Industry,Hacktype
765,765,"March 2013. The Syrian Electronic Army, a pro ...",172,0,9,0
86,86,January 2022. A Chinese hacking group breached...,36,66,1,2
46,46,April 2022. Iranian state tele vision claimed ...,0,81,6,0
726,726,September 2014. A false Occupy Central smartph...,36,76,0,4
898,898,"September 2007. Francis D elon, Secretary -Gen...",36,62,2,0


In [ ]:
ht_extra = pd.read_csv('/content/cyber_incidents_w_labels_extra_train.csv', index_col=0)
ht_extra.head()

,Unnamed: 0.1,Description,hacktype_label
823,823,"April 2011. Between March 2010 and April 2011,...",0
218,218,January 2021. Unidentified hackers breached on...,2
111,111,October 2021. A cyberattack targeted the gover...,0
48,48,April 2022. Hackers targeted members of the Eu...,4
283,283,August 2020 . A North Korean hacking group tar...,5


In [ ]:
ht_train_labels_all = pd.concat([ht_train['Hacktype'], ht_extra['hacktype_label']])
ht_train_labels_all

765    0
86     2
46     0
726    4
898    0
      ..
25     1
739    2
635    2
302    0
173    4
Length: 200, dtype: int64

In [ ]:
ht_data = pd.read_csv('/content/ht_w_unseen_labels.csv', index_col=0)

In [ ]:
ht_data['Hacktype'] = ht_train_labels_all

In [ ]:
ht_data.head()

,Description,hacktype_class,hacktype,Unnamed: 0.1,Description.1,hacker_label,victim_label,industry_label,hacktype_label,Hacktype
0,September 2022. Iranian hackers targeted Alban...,4,Malware,NaN,NaN,NaN,NaN,NaN,NaN,4.0
1,September 2022. Hackers targeted Montenegroâ...,1,Denial of Service,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,September 2022. Hackers targeted the state -le...,1,Denial of Service,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,September 2022. China accused the U.S. Nationa...,2,Data Exfiltration,NaN,NaN,NaN,NaN,NaN,NaN,2.0
4,September 2022. The group Anonymous took respo...,0,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
ht = ht_data[['hacktype_class']].copy()
ht.head()

,hacktype_class
0,4
1,1
2,1
3,2
4,0


In [ ]:
# train and test labels
ht_data[ht_data['hacktype_class'] != ht_data['Hacktype']][['hacktype_class', 'Hacktype']].dropna()

,hacktype_class,Hacktype
128,0,2.0
212,2,5.0
299,2,1.0
312,0,7.0
465,0,2.0
525,2,0.0
700,0,4.0
719,0,2.0
750,3,2.0
894,0,5.0


In [ ]:
# unseen labels
ht_data[ht_data['hacktype_class'] != ht_data['hacktype_label']][['hacktype_class', 'hacktype_label']].dropna()

,hacktype_class,hacktype_label
171,0,3.0
231,2,7.0
493,4,2.0
641,0,1.0
654,0,2.0
748,1,5.0
801,4,0.0
863,0,2.0
897,0,7.0
910,0,2.0


In [ ]:
# update values to labeled data
ht.loc[ht_data['Hacktype'].notnull()] = ht_data[['Hacktype']].dropna()   # train and test
ht.loc[ht_data['hacktype_label'].notnull()] = ht_data[['hacktype_label']].dropna() # unseen
# ht

In [ ]:
(ht.loc[ht_data['Hacktype'].notnull()]['hacktype_class'] != ht_data['Hacktype'].dropna()).sum()

0

In [ ]:
(ht.loc[ht_data['hacktype_label'].notnull()]['hacktype_class'] != ht_data['hacktype_label'].dropna()).sum()

0

### Convert hacktype class to name

In [ ]:
ht_list = np.load('hacktype_list.npy')

int_to_ht = {}

for i, hacktype in enumerate(ht_list):
  int_to_ht[i] = hacktype

In [ ]:
ht['hacktype'] = ht['hacktype_class'].apply(lambda x: int_to_ht[x])

In [ ]:
ht.head()

,hacktype_class,hacktype
0,4.0,Malware
1,1.0,Denial of Service
2,1.0,Denial of Service
3,2.0,Data Exfiltration
4,0.0,Unknown


## Combine All Results

In [ ]:
output = pd.read_csv('/content/cybersecurity_incidents_descriptions_v2.csv', encoding="ISO-8859-1", index_col=0)
output

,Description
0,September 2022. Iranian hackers targeted Alban...
1,September 2022. Hackers targeted Montenegroâ...
2,September 2022. Hackers targeted the state -le...
3,September 2022. China accused the U.S. Nationa...
4,September 2022. The group Anonymous took respo...
...,...
909,August 2006. A senior A ir Force Officer state...
910,May 2006. The Department of Stateâs networks...
911,April 2005 . Chinese hackers infiltrated NASA ...
912,January 2005. Chinese hackers infiltrated U.S....


In [ ]:
output = pd.concat([
   output, 
   date_labels[['month', 'year']], 
   hacker_origin[['country']], 
   victim_origin[['country']], 
   industry[['industry']],
   ht[['hacktype']]
], axis=1)

In [ ]:
output

,Description,month,year,country,country,industry,hacktype
0,September 2022. Iranian hackers targeted Alban...,September,2022,Iran,Albania,Government,Malware
1,September 2022. Hackers targeted Montenegroâ...,September,2022,Russia,Montenegro,Government,Denial of Service
2,September 2022. Hackers targeted the state -le...,September,2022,Unknown,Bosnia and Herzegovina,Government,Denial of Service
3,September 2022. China accused the U.S. Nationa...,September,2022,United States,China,Education,Data Exfiltration
4,September 2022. The group Anonymous took respo...,September,2022,Unknown,Iran,Government,Unknown
...,...,...,...,...,...,...,...
909,August 2006. A senior A ir Force Officer state...,August,2006,Unknown,United States,Defense,Malware
910,May 2006. The Department of Stateâs networks...,May,2006,Unknown,United States,Government,Data Exfiltration
911,April 2005 . Chinese hackers infiltrated NASA ...,April,2005,China,United States,Government,Data Exfiltration
912,January 2005. Chinese hackers infiltrated U.S....,January,2005,China,United States,Defense,Unknown


In [ ]:
output.to_csv('cybersecurity_incidents_final.csv')